In [3]:
# Install required packages
!pip install litellm transformers datasets

# Import necessary libraries
import os
import subprocess
import sys
import tempfile
from datasets import load_dataset

print("Dependencies installed and libraries imported successfully.")

Dependencies installed and libraries imported successfully.


In [ ]:
# Load the dataset from Hugging Face
dataset = load_dataset("isavita/advent-of-code")

# Filter the dataset for Go solutions
go_solutions = dataset['train'].filter(lambda example: example['solution_lang'] == 'go')

print(f"Loaded {len(go_solutions)} Go solutions from the dataset.")

In [24]:
class Constants:
    @staticmethod
    def file_ext():
        return "jl"
    
    @staticmethod
    def lang():
        return "julia"
    
    @staticmethod
    def compiler():
        return "julia"
    
    @staticmethod
    def runtime():
        return "/opt/homebrew/bin/julia"

print("Go constants class defined.")

Go constants class defined.


In [6]:
import tempfile
import subprocess
import os

def test_go_execution():
    # Hello World program in Go
    code = '''
print("Hello, World!")
'''
    
    # Create a temporary file
    with tempfile.NamedTemporaryFile(suffix=f'.{Constants.file_ext()}', delete=False, mode='w') as temp_file:
        temp_file.write(code)
        temp_file_path = temp_file.name
    
    try:
        # Run the program
        result = subprocess.run(f"{Constants.runtime()} {temp_file_path}", shell=True, capture_output=True, text=True)
        
        if result.returncode == 0:
            print("Program executed successfully.")
            print("Output:", result.stdout.strip())
        else:
            print("Error executing program:")
            print(result.stderr)
    finally:
        # Clean up the temporary file
        os.unlink(temp_file_path)

# Run the test
test_go_execution()

Program executed successfully.
Output: Hello, World!


In [25]:
import litellm
import warnings

# Suppress the specific warning
warnings.filterwarnings("ignore", message="Field .* has conflict with protected namespace .*")

class LLMWrapper:
    def __init__(self, model_name, api_base=None):
        self.model_name = model_name
        self.api_base = api_base
        self.temperature = 0.3  # Hardcoded temperature as specified

    def generate(self, system_prompt, user_prompt):
        try:
            completion_params = {
                "model": self.model_name,
                "messages": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                "temperature": self.temperature,
            }
            
            if self.api_base:
                completion_params["api_base"] = self.api_base

            response = litellm.completion(**completion_params)
            return response.choices[0].message.content
        except Exception as e:
            print(f"Error in LLM API call: {str(e)}")
            return None

# Test the LLMWrapper with the local Ollama model
def test_llm_wrapper():
    wrapper = LLMWrapper("ollama/llama3:8b-instruct-q5_K_M", api_base="http://localhost:11434")
    
    system_prompt = "You are a helpful assistant."
    user_prompt = "Tell me a short joke about programming."
    
    print("Testing with local Ollama model:")
    response = wrapper.generate(system_prompt, user_prompt)
    if response:
        print("Local LLM Response:")
        print(response)
    else:
        print("Failed to get a response from the local LLM.")

# Run the test
test_llm_wrapper()

Testing with local Ollama model:
Local LLM Response:
Here's one:

Why do programmers prefer dark mode?

Because light attracts bugs!

Hope that made you laugh! Do you have any other questions or requests?


In [31]:
import threading
import time
import queue

class TimeoutRunner:
    @staticmethod
    def run_with_timeout(func, args=(), kwargs={}, timeout_duration=10):
        result_queue = queue.Queue()

        def wrapper():
            try:
                result = func(*args, **kwargs)
                result_queue.put(result)
            except Exception as e:
                result_queue.put(e)

        thread = threading.Thread(target=wrapper)
        thread.daemon = True
        thread.start()
        
        try:
            return result_queue.get(timeout=timeout_duration)
        except queue.Empty:
            return None

# Test the TimeoutRunner
def test_timeout_runner():
    def quick_function():
        return "Quick function completed"

    def slow_function():
        time.sleep(5)
        return "Slow function completed"

    runner = TimeoutRunner()

    print("Testing quick function (should complete):")
    result = runner.run_with_timeout(quick_function, timeout_duration=2)
    print(result)

    print("\nTesting slow function (should timeout):")
    result = runner.run_with_timeout(slow_function, timeout_duration=2)
    print(result)

# Run the test
test_timeout_runner()

Testing quick function (should complete):
Quick function completed

Testing slow function (should timeout):
None


In [32]:
import os
import tempfile
import subprocess

class EvaluateCode:
    def __init__(self, code, input_data):
        self.code = code
        self.input_data = input_data

    def run(self):
        with tempfile.TemporaryDirectory() as temp_dir:
            # Save code to a file using the extension from Constants
            code_file = os.path.join(temp_dir, f"main.{Constants.file_ext()}")
            with open(code_file, 'w') as f:
                f.write(self.code)

            # Save input to a file
            input_file = os.path.join(temp_dir, "input.txt")
            with open(input_file, 'w') as f:
                f.write(self.input_data)

            # Directly run the file
            run_command = f"{Constants.runtime()} {code_file} < {input_file}"

            # Run the command in the temporary directory
            run_result = subprocess.run(
                run_command,
                shell=True,
                capture_output=True,
                text=True,
                cwd=temp_dir  # Set the working directory to the temporary directory
            )

            if run_result.returncode != 0:
                return f"Runtime error: {run_result.stderr}"

            return run_result.stdout

# Test the EvaluateCode class
def test_evaluate_code():
    code = '''
print("Enter a number:")
n = io.read("*n")
print("The number is: " .. n)
'''
    input_data = "43"

    evaluator = EvaluateCode(code, input_data)
    result = evaluator.run()
    print("Evaluation result:")
    print(result)

# Run the test
test_evaluate_code()

Evaluation result:
Runtime error: ERROR: LoadError: UndefVarError: `io` not defined
Stacktrace:
 [1] top-level scope
   @ /var/folders/46/6lz6t86n4n10lwsyj3pyzd6h0000gn/T/tmpdskqkkt1/main.jl:3
in expression starting at /var/folders/46/6lz6t86n4n10lwsyj3pyzd6h0000gn/T/tmpdskqkkt1/main.jl:3



In [29]:
import re
import os
from pathlib import Path

class ChallengeSolver:
    def __init__(self, model_name, llm_wrapper, language='julia'):
        self.model_name = model_name
        self.llm_wrapper = llm_wrapper
        self.language = language

    def extract_code(self, input_text):
        pattern = rf"```(?:{self.language})?(.*?)```"
        matches = re.findall(pattern, input_text, re.DOTALL)
        return matches[0] if matches else input_text

    def valid_solution(self, result, answer):
        special_cases = [
            ".##..####.###..#..#.###..####.###....##.###...###.",
            " ##  #### ###  #  # ###  #### ###    ## ###   ### ",
            "#....#..#....#.....###..######....##....#....#....##....######",
            "#    #  #    #     ###  ######    ##    #    #    ##    ######",
            "3.465154e+06", "3.465154e+6",
            "####.###..####.#..#.###..\n#....#..#....#.#..#.#..#.",
            "#### ###  #### #  # ###  \n#    #  #    # #  # #  # ",
            ".#....###....##.#..#.####.#..#.#....#..#.\n",
            " #    ###    ## #  # #### #  # #    #  # \n",
            " █    ███    ██ █  █ ████ █  █ █    █  █ \n",
            "#..#.#..#.#..#.#..#.#..#.#..#.#..#....#",
            "#  # #  # #  # #  # #  # #  # #  #    #",
            "###..###..###...##..###...##...##..####.",
            "###  ###  ###   ##  ###   ##   ##  #### "
        ]
        return answer in result or any(case in result for case in special_cases)

    def solve(self, challenge_data, use_go_solution=False):
        if use_go_solution:
            system_prompt = f"""
            You are an expert programmer that writes simple, concise code and no comments or explanation.
            Given this golang program write a {self.language} version that it reads its input from a file "input.txt" and solve the following task.
            The program should print the answer.

            **NOTE:** Golang is faster language than {self.language}, thus you have to try more efficient solution in {self.language}!

            Golang Solution:
            {challenge_data['solution']}
            """
            user_prompt = f"Convert the above Go solution to {self.language} and optimize it if possible."
        else:
            system_prompt = f"""
            Write a {self.language} program that reads input from a file called input.txt and prints the output to standard output.
            Focus on writing clean, efficient code that demonstrates your programming skills by concisely solving the challenge.
            {"**NOTE:** DO NOT forget to add at the top of the code #!/usr/bin/awk" if self.language == 'awk' else ""}

            Coding challenge:
            """
            user_prompt = challenge_data['task']

        max_attempts = 1
        for attempt in range(max_attempts):
            response = self.llm_wrapper.generate(system_prompt, user_prompt)
            code = self.extract_code(response)

            evaluator = EvaluateCode(code, challenge_data['input'])
            result = evaluator.run()

            if self.valid_solution(result, challenge_data['answer']):
                self.save_solution(challenge_data, code)
                return True, result

        return False, "Failed to find a valid solution after multiple attempts."

    def save_solution(self, challenge_data, code):
        home_dir = str(Path.home())
        file_path = f"{home_dir}/code/advent_generated/{challenge_data['name']}.{self.get_file_extension()}"
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        with open(file_path, 'w') as f:
            f.write(code)

    def get_file_extension(self):
        extensions = {
            'go': 'go',
            'python': 'py',
            'awk': 'awk',
            'zig': 'zig',
            'ruby': 'rb',
            'lua': 'lua',
            'julia': 'jl',
            # Add more languages and their extensions as needed
        }
        return extensions.get(self.language, '')

# Example usage and test
def test_challenge_solver():
    class MockLLMWrapper:
        def generate(self, system_prompt, user_prompt):
            return '''Here's a Python solution for the problem:

```python
#!/usr/bin/env python3
with open('input.txt', 'r') as f:
    data = f.read().strip()

sum = 0
for i in range(len(data)):
    if data[i] == data[(i + 1) % len(data)]:
        sum += int(data[i])

print(sum)
```'''

    # Sample challenge data
    challenge_data = {
        "name": "day1_part1_2017",
        "task": "Find the sum of all digits that match the next digit in the list. The list is circular, so the digit after the last digit is the first digit in the list.",
        "input": "1122",
        "answer": "3",
        "solution": "package main\n\nimport (\n\t\"fmt\"\n\t\"os\"\n\t\"strings\"\n)\n\nfunc main() {\n\tdata, _ := os.ReadFile(\"input.txt\")\n\tinput := strings.TrimSpace(string(data))\n\tsum := 0\n\tfor i := 0; i < len(input); i++ {\n\t\tif input[i] == input[(i+1)%len(input)] {\n\t\t\tsum += int(input[i] - '0')\n\t\t}\n\t}\n\tfmt.Println(sum)\n}",
        "solution_lang": "go",
        "year": 2017
    }

    solver = ChallengeSolver("gpt-3.5-turbo", MockLLMWrapper(), language='python')
    
    print("Testing with task description:")
    success, result = solver.solve(challenge_data, use_go_solution=False)
    print(f"Solution found: {success}")
    print(f"Result: {result}")

    print("\nTesting with Go solution conversion:")
    success, result = solver.solve(challenge_data, use_go_solution=True)
    print(f"Solution found: {success}")
    print(f"Result: {result}")

# Run the test
test_challenge_solver()

Testing with task description:
Solution found: True
Result: Runtime error: ERROR: LoadError: UndefVarError: `with` not defined
in expression starting at /var/folders/46/6lz6t86n4n10lwsyj3pyzd6h0000gn/T/tmpsefl1s9f/main.jl:3


Testing with Go solution conversion:
Solution found: True
Result: Runtime error: ERROR: LoadError: UndefVarError: `with` not defined
in expression starting at /var/folders/46/6lz6t86n4n10lwsyj3pyzd6h0000gn/T/tmpvtgjxy8s/main.jl:3



In [42]:
import os
import time
from datasets import load_dataset
from pathlib import Path

def get_existing_solutions(language):
    home_dir = str(Path.home())
    solutions_dir = os.path.join(home_dir, "code", "advent_generated", "julia")
    solutions = os.listdir(solutions_dir)
    return [os.path.splitext(solution)[0] for solution in solutions if solution.endswith(f".{language}")]

def run_max_times(func, max_attempts=1):
    for _ in range(max_attempts):
        success, result = func()
        if success:
            return True, result
    return False, "Failed after maximum attempts"

def main():
    target_language = 'julia'  # Change this to 'awk' or any other language as needed

    dataset = load_dataset("isavita/advent-of-code")
    go_solutions = dataset['train'].filter(lambda example: example['solution_lang'] == 'go')

    existing_solutions = get_existing_solutions('jl')
    unsolved_challenges = [challenge for challenge in go_solutions if challenge['name'] not in existing_solutions]

    llm_wrapper = LLMWrapper("gpt-4o-mini")  # You can change this to your preferred model
    solver = ChallengeSolver("gpt-4o-mini", llm_wrapper, language=target_language)

    total_challenges = len(unsolved_challenges)
    print(f"Total unsolved challenges: {total_challenges}")

    for i, challenge in enumerate(unsolved_challenges[:], 1):
        def solve_attempt():
            success, result = solver.solve(challenge, use_go_solution=True)
            print(result)
            if "Timeout" in result:
                print(f"Challenge {challenge['name']} timed out.")
                return False, result
            return success, result

        success, result = run_max_times(solve_attempt, max_attempts=1)

        if success:
            print(f"Solved {challenge['name']} successfully!")
        else:
            print(f"Failed to solve {challenge['name']}: {result}")

        progress = (i / total_challenges) * 100
        print(f"Progress: {progress:.1f}%")
        
        time.sleep(1)  # Sleep for 15 seconds between attempts

if __name__ == "__main__":
    main()

Total unsolved challenges: 146
1342

Solved day8_part1_2015 successfully!
Progress: 0.7%
Failed to find a valid solution after multiple attempts.
Failed to solve day10_part1_2015: Failed after maximum attempts
Progress: 1.4%
Failed to find a valid solution after multiple attempts.
Failed to solve day15_part2_2015: Failed after maximum attempts
Progress: 2.1%
Failed to find a valid solution after multiple attempts.
Failed to solve day16_part1_2015: Failed after maximum attempts
Progress: 2.7%
Failed to find a valid solution after multiple attempts.
Failed to solve day7_part2_2016: Failed after maximum attempts
Progress: 3.4%
Failed to find a valid solution after multiple attempts.
Failed to solve day11_part1_2016: Failed after maximum attempts
Progress: 4.1%
Failed to find a valid solution after multiple attempts.
Failed to solve day11_part2_2016: Failed after maximum attempts
Progress: 4.8%
Failed to find a valid solution after multiple attempts.
Failed to solve day21_part1_2016: Faile

KeyboardInterrupt: 